# Hyperparameter Tuning for Phishing Detection Models

Optuna-based hyperparameter optimization for:
- TF-IDF + Logistic Regression
- XGBoost Hybrid
- MLP Hybrid

**Objective:** Maximize Recall + ROC-AUC (catch phishing + good ranking)

## Setup and Imports

In [28]:
import os
import sys
import json
import pickle
from datetime import datetime
from pathlib import Path
import numpy as np
import polars as pl
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
     precision_score, recall_score, f1_score, roc_auc_score,
)
from sentence_transformers import SentenceTransformer
import xgboost as xgb
import optuna
from optuna.integration.wandb import WeightsAndBiasesCallback

sys.path.append(str(Path.cwd().parent.parent))

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Device: {'cuda' if torch.cuda.is_available() else 'cpu'}")
print(f"Optuna version: {optuna.__version__}")

PyTorch version: 2.5.1+cu121
CUDA available: True
Device: cuda
Optuna version: 4.6.0


## Configuration

In [29]:
config = {
    "random_seed": 42,
    "data_source": "../../emails_v6.parquet",
    "numeric_features": [
        "sender_domain_entropy", "has_attachment",
        "spf_flag_missing", "dkim_flag_missing",
        "num_links", "subject_length", "body_length", "keyword_count", 
        "num_received_headers", "num_exclamation_marks", 
        "num_malicious_links"
    ],
    "embedding_model": "all-MiniLM-L6-v2",
    "embedding_dim": 384,
    "n_trials_tfidf": 30,
    "n_trials_xgb": 30,
    "n_trials_mlp": 30
}

In [30]:

df = pl.read_parquet("../../emails_v6.parquet")
from app.data_processing.preprocessing_pipeline import PreprocessingPipeline

df = PreprocessingPipeline().process_pipeline(df)
external_df = df.filter(
    (pl.col("source") == "data/csv_misc/TREC-07.csv") | (pl.col("source") == "phishing-2020")
)

train_pool_df = df.filter(
    (pl.col("source") != "data/csv_misc/TREC-07.csv") & (pl.col("source") != "phishing-2020")
)

train_pool_pd = train_pool_df.to_pandas()

train_val_df_pd, test_df_pd = train_test_split(
    train_pool_pd, 
    test_size=0.2, 
    random_state=config["random_seed"], 
    stratify=train_pool_pd['phishing']
)
train_df_pd, val_df_pd = train_test_split(
    train_val_df_pd,
    test_size=0.2,  
    random_state=config["random_seed"],
    stratify=train_val_df_pd['phishing']
)

train_df = pl.from_pandas(train_df_pd)
val_df = pl.from_pandas(val_df_pd)
test_df = pl.from_pandas(test_df_pd)

print(f"\nFinal splits:")
print(f"  Train: {len(train_df):,} ({len(train_df)/len(df)*100:.1f}%)")
print(f"  Validation: {len(val_df):,} ({len(val_df)/len(df)*100:.1f}%) ")
print(f"  Test: {len(test_df)} ({len(test_df)/len(df)*100:.1f}%)")
print(f"  External: {len(external_df)} ({len(external_df)/len(df)*100:.1f}%")

for name, split_df in [("Train", train_df), ("Validation", val_df), ("Test", test_df), ("External", external_df)]:
    phish = split_df.filter(pl.col("phishing") == 1).shape[0]
    total = len(split_df)
    print(f"  {name}: {phish}/{total} phishing ({phish/total*100:.1f}%)")

numeric_features = config["numeric_features"]

features_train = train_df.select(numeric_features).fill_null(0).to_numpy()
features_val = val_df.select(numeric_features).fill_null(0).to_numpy()
features_test = test_df.select(numeric_features).fill_null(0).to_numpy()

y_train = train_df['phishing'].to_numpy()
y_val = val_df['phishing'].to_numpy()
y_test = test_df['phishing'].to_numpy()


Final splits:
  Train: 101,649 (47.9%)
  Validation: 25,413 (12.0%) 
  Test: 31766 (15.0%)
  External: 53285 (25.1%
  Train: 42291/101649 phishing (41.6%)
  Validation: 10573/25413 phishing (41.6%)
  Test: 13216/31766 phishing (41.6%)
  External: 28983/53285 phishing (54.4%)


## Generate Embeddings

In [31]:
device = "cuda" if torch.cuda.is_available() else "cpu"
embedding_model = SentenceTransformer(config["embedding_model"], device=device)

X_emb_train = embedding_model.encode(
    train_df['body_subject'].to_list(), 
    show_progress_bar=True, 
    convert_to_numpy=True,
    batch_size=32
)

X_emb_val = embedding_model.encode(
    val_df['body_subject'].to_list(), 
    show_progress_bar=True, 
    convert_to_numpy=True,
    batch_size=32
)

X_emb_test = embedding_model.encode(
    test_df['body_subject'].to_list(), 
    show_progress_bar=True, 
    convert_to_numpy=True,
    batch_size=32
)

print(f"  Train: {X_emb_train.shape}")
print(f"  Validation: {X_emb_val.shape}")
print(f"  Test: {X_emb_test.shape}")

'(ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), '(Request ID: ae26560a-196c-4d96-8562-28ff56dd866c)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].
Retrying in 1s [Retry 1/5].


Batches: 100%|██████████| 3177/3177 [02:52<00:00, 18.41it/s] 

Batches: 100%|██████████| 795/795 [00:47<00:00, 16.64it/s] 

Batches: 100%|██████████| 993/993 [00:57<00:00, 17.21it/s] 



  Train: (101649, 384)
  Validation: (25413, 384)
  Test: (31766, 384)


## TF-IDF Tuning

In [32]:
from scipy.sparse import hstack

def tfidf_objective(trial):
    params = {
        'max_features': trial.suggest_int('max_features', 3000, 10000, step=1000),
        'min_df': trial.suggest_int('min_df', 2, 10),
        'max_df': trial.suggest_float('max_df', 0.5, 0.95, step=0.05),
        'ngram_max': trial.suggest_int('ngram_max', 1, 3),
        'C': trial.suggest_float('C', 0.1, 10.0, log=True),
        'class_weight_ratio': trial.suggest_float('class_weight_ratio', 1.0, 10.0, step=0.5),
        'threshold': trial.suggest_float('threshold', 0.2, 0.5, step=0.05)
    }
    
    vectorizer = TfidfVectorizer(
        max_features=params['max_features'],
        ngram_range=(1, params['ngram_max']),
        min_df=params['min_df'],
        max_df=params['max_df']
    )

    X_tfidf_train = vectorizer.fit_transform(train_df['body_subject'].to_list())
    X_tfidf_val = vectorizer.transform(val_df['body_subject'].to_list())
    
    X_combined_train = hstack([X_tfidf_train, features_train])
    X_combined_val = hstack([X_tfidf_val, features_val])
    
    clf = LogisticRegression(
        max_iter=1000,
        C=params['C'],
        class_weight={0: 1.0, 1: params['class_weight_ratio']},
        random_state=42,
        verbose=0
    )
    clf.fit(X_combined_train, y_train)
    
    y_proba = clf.predict_proba(X_combined_val)[:, 1]
    y_pred = (y_proba >= params['threshold']).astype(int)
    
    recall = recall_score(y_val, y_pred, zero_division=0)
    precision = precision_score(y_val, y_pred, zero_division=0)
    f1 = f1_score(y_val, y_pred, zero_division=0)
    roc_auc = roc_auc_score(y_val, y_proba)
    
    trial.set_user_attr('recall', recall)
    trial.set_user_attr('precision', precision)
    trial.set_user_attr('f1_score', f1)
    trial.set_user_attr('roc_auc', roc_auc)
    
    if precision + recall > 0:
        F2 = (5 * precision * recall) / (4 * precision + recall)
    else:
        F2 = 0
    return F2

In [33]:
wandb_callback = WeightsAndBiasesCallback(
    metric_name="objective_score",
    wandb_kwargs={"project": "phishstop-detection", "tags": ["optuna", "tfidf"]}
)

tfidf_study = optuna.create_study(
    direction="maximize",
    study_name="tfidf-tuning",
    sampler=optuna.samplers.TPESampler(seed=42)
)

tfidf_study.optimize(
    tfidf_objective,
    n_trials=config["n_trials_tfidf"],
    callbacks=[wandb_callback],
    show_progress_bar=True
)

print(f"Best score: {tfidf_study.best_trial.value:.4f}")
print(f"Recall: {tfidf_study.best_trial.user_attrs['recall']:.4f}")
print(f"ROC-AUC: {tfidf_study.best_trial.user_attrs['roc_auc']:.4f}")
print(f"Precision: {tfidf_study.best_trial.user_attrs['precision']:.4f}")
print(f"F1: {tfidf_study.best_trial.user_attrs['f1_score']:.4f}")
print("Best params:")
for k, v in tfidf_study.best_trial.params.items():
    print(f"  {k}: {v}")

C:\Users\Latandu\AppData\Local\Temp\ipykernel_20948\3685201085.py:1: ExperimentalWarning:

WeightsAndBiasesCallback is experimental (supported from v2.9.0). The interface can change in the future.



[I 2025-12-01 00:52:52,190] A new study created in memory with name: tfidf-tuning
Best trial: 0. Best value: 0.948664:   3%|▎         | 1/30 [01:00<29:02, 60.07s/it]

[I 2025-12-01 00:53:52,258] Trial 0 finished with value: 0.9486643354122354 and parameters: {'max_features': 5000, 'min_df': 10, 'max_df': 0.8500000000000001, 'ngram_max': 2, 'C': 0.20513382630874505, 'class_weight_ratio': 2.0, 'threshold': 0.2}. Best is trial 0 with value: 0.9486643354122354.


Best trial: 1. Best value: 0.97625:   7%|▋         | 2/30 [01:20<17:13, 36.92s/it] 

[I 2025-12-01 00:54:12,979] Trial 1 finished with value: 0.9762501159662305 and parameters: {'max_features': 9000, 'min_df': 7, 'max_df': 0.8500000000000001, 'ngram_max': 1, 'C': 8.706020878304859, 'class_weight_ratio': 8.5, 'threshold': 0.25}. Best is trial 1 with value: 0.9762501159662305.


...

[I 2025-12-01 01:11:40,418] Trial 27 finished with value: 0.9790275030787028 and parameters: {'max_features': 10000, 'min_df': 8, 'max_df': 0.9, 'ngram_max': 2, 'C': 5.537277379395257, 'class_weight_ratio': 8.5, 'threshold': 0.45}. Best is trial 27 with value: 0.9790275030787028.


Best trial: 27. Best value: 0.979028:  97%|█████████▋| 29/30 [19:41<00:42, 42.71s/it]

[I 2025-12-01 01:12:33,191] Trial 28 finished with value: 0.9741210032973956 and parameters: {'max_features': 9000, 'min_df': 8, 'max_df': 0.9, 'ngram_max': 2, 'C': 1.201179592539979, 'class_weight_ratio': 9.0, 'threshold': 0.45}. Best is trial 27 with value: 0.9790275030787028.


Best trial: 27. Best value: 0.979028: 100%|██████████| 30/30 [23:03<00:00, 46.12s/it]

[I 2025-12-01 01:15:55,755] Trial 29 finished with value: 0.9741812318128741 and parameters: {'max_features': 6000, 'min_df': 10, 'max_df': 0.9, 'ngram_max': 3, 'C': 2.816469084419117, 'class_weight_ratio': 9.5, 'threshold': 0.45}. Best is trial 27 with value: 0.9790275030787028.
Best score: 0.9790
Recall: 0.9925
ROC-AUC: 0.9973
Precision: 0.9285
F1: 0.9595
Best params:
  max_features: 10000
  min_df: 8
  max_df: 0.9
  ngram_max: 2
  C: 5.537277379395257
  class_weight_ratio: 8.5
  threshold: 0.45


In [34]:
fig = optuna.visualization.plot_optimization_history(tfidf_study)
fig.update_layout(width=800, height=500) 
fig.show()

fig = optuna.visualization.plot_param_importances(tfidf_study)
fig.update_layout(width=800, height=500) 
fig.show()

## XGBoost Tuning

In [35]:
def xgboost_objective(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'n_estimators': trial.suggest_int('n_estimators', 100, 500, step=50),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0, step=0.05),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0, step=0.05),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1.0, 10.0, step=0.5),
        'threshold': trial.suggest_float('threshold', 0.2, 0.5, step=0.05)
    }
    
    X_train = np.concatenate([X_emb_train, features_train], axis=1)
    X_val = np.concatenate([X_emb_val, features_val], axis=1)
    
    clf = xgb.XGBClassifier(
        max_depth=params['max_depth'],
        n_estimators=params['n_estimators'],
        learning_rate=params['learning_rate'],
        min_child_weight=params['min_child_weight'],
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        scale_pos_weight=params['scale_pos_weight'],
        random_state=42,
        eval_metric='logloss',
        early_stopping_rounds=10,
        verbosity=0
    )
    clf.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    y_proba = clf.predict_proba(X_val)[:, 1]
    y_pred = (y_proba >= params['threshold']).astype(int)
    
    recall = recall_score(y_val, y_pred, zero_division=0)
    precision = precision_score(y_val, y_pred, zero_division=0)
    f1 = f1_score(y_val, y_pred, zero_division=0)
    roc_auc = roc_auc_score(y_val, y_proba)
    
    trial.set_user_attr('recall', recall)
    trial.set_user_attr('precision', precision)
    trial.set_user_attr('f1_score', f1)
    trial.set_user_attr('roc_auc', roc_auc)
    
    if precision + recall > 0:
        F2 = (5 * precision * recall) / (4 * precision + recall)
    else:
        F2 = 0
    return F2

In [36]:
wandb_callback_xgb = WeightsAndBiasesCallback(
    metric_name="objective_score",
    wandb_kwargs={"project": "phishstop-detection", "tags": ["optuna", "xgboost"]}
)

xgb_study = optuna.create_study(
    direction="maximize",
    study_name="xgboost-tuning",
    sampler=optuna.samplers.TPESampler(seed=42)
)

xgb_study.optimize(
    xgboost_objective,
    n_trials=config["n_trials_xgb"],
    callbacks=[wandb_callback_xgb],
    show_progress_bar=True
)

print(f"Best score: {xgb_study.best_trial.value:.4f}")
print(f"Recall: {xgb_study.best_trial.user_attrs['recall']:.4f}")
print(f"ROC-AUC: {xgb_study.best_trial.user_attrs['roc_auc']:.4f}")
print(f"Precision: {xgb_study.best_trial.user_attrs['precision']:.4f}")
print(f"F1: {xgb_study.best_trial.user_attrs['f1_score']:.4f}")
print("Best params:")
for k, v in xgb_study.best_trial.params.items():
    print(f"  {k}: {v}")

C:\Users\Latandu\AppData\Local\Temp\ipykernel_20948\2868440370.py:1: ExperimentalWarning:

WeightsAndBiasesCallback is experimental (supported from v2.9.0). The interface can change in the future.



C,▁▇▁▄▁▁▁▅▁▄▂▇▇▂▂█▄▄▂▅▅▅▅▃▅▂▃▅▂▃
class_weight_ratio,▁▇▃▂█▁▄▅▃▁▇▆▆▆▆█▅▄▃▅▇▇▇▇█▆▇▇▇█
max_df,▆▆▃▃▆▁▃▆▁▅█▃█▄▆▃▅▅▇▆▆▆▆▆▄▇▆▇▇▇
max_features,▃▇▂▂▅▇▁▂▁▃█▆▆▆█▅▇▇▇██████▆▄█▇▄
min_df,█▅▂▃▁▃██▂▃▅▅▅▆▅▆▅▅▄▆▇▇▇▇▇▇▇▆▆█
ngram_max,▅▁▅▅▁█▅█▁▁▁▅█▁▅▅▁▁▁▁▁▁▁▁▁▁▁▅▅█
objective_score,▁▇▆▇▄▇▄▇▆▇▅▇▇▆▅▇▇▇▇████▇█▇▇█▇▇
threshold,▁▂▆▄█▄▄█▇█▁▂▂▂▂▃▃▃▃▆▆▆▆▆▇▇▆▇▇▇
C,2.81647
class_weight_ratio,9.5
max_df,0.9


[I 2025-12-01 01:16:18,461] A new study created in memory with name: xgboost-tuning
Best trial: 0. Best value: 0.972406:   3%|▎         | 1/30 [00:37<18:17, 37.84s/it]

[I 2025-12-01 01:16:56,299] Trial 0 finished with value: 0.9724062227942014 and parameters: {'max_depth': 6, 'n_estimators': 500, 'learning_rate': 0.1205712628744377, 'min_child_weight': 6, 'subsample': 0.65, 'colsample_bytree': 0.65, 'scale_pos_weight': 1.5, 'threshold': 0.5}. Best is trial 0 with value: 0.9724062227942014.


Best trial: 0. Best value: 0.972406:   7%|▋         | 2/30 [01:55<28:34, 61.23s/it]

...

[I 2025-12-01 01:34:02,869] Trial 27 finished with value: 0.9714099459862172 and parameters: {'max_depth': 5, 'n_estimators': 250, 'learning_rate': 0.15183422653050652, 'min_child_weight': 9, 'subsample': 0.65, 'colsample_bytree': 0.8, 'scale_pos_weight': 7.0, 'threshold': 0.4}. Best is trial 24 with value: 0.9770493035845189.


Best trial: 24. Best value: 0.977049:  97%|█████████▋| 29/30 [18:33<00:40, 40.50s/it]

[I 2025-12-01 01:34:52,022] Trial 28 finished with value: 0.9760556513912849 and parameters: {'max_depth': 7, 'n_estimators': 450, 'learning_rate': 0.059869800338550265, 'min_child_weight': 8, 'subsample': 0.85, 'colsample_bytree': 0.95, 'scale_pos_weight': 8.5, 'threshold': 0.45}. Best is trial 24 with value: 0.9770493035845189.


Best trial: 24. Best value: 0.977049: 100%|██████████| 30/30 [19:18<00:00, 38.61s/it]

[I 2025-12-01 01:35:36,815] Trial 29 finished with value: 0.9769107529921778 and parameters: {'max_depth': 6, 'n_estimators': 500, 'learning_rate': 0.1022268832486803, 'min_child_weight': 6, 'subsample': 0.7, 'colsample_bytree': 1.0, 'scale_pos_weight': 6.0, 'threshold': 0.5}. Best is trial 24 with value: 0.9770493035845189.
Best score: 0.9770
Recall: 0.9833
ROC-AUC: 0.9974
Precision: 0.9530
F1: 0.9679
Best params:
  max_depth: 7
  n_estimators: 400
  learning_rate: 0.0846751074136459
  min_child_weight: 7
  subsample: 0.7
  colsample_bytree: 0.95
  scale_pos_weight: 7.0
  threshold: 0.45


In [37]:
fig = optuna.visualization.plot_optimization_history(xgb_study)
fig.update_layout(width=800, height=500) 
fig.show()

fig = optuna.visualization.plot_param_importances(xgb_study)
fig.update_layout(width=800, height=500) 
fig.show()

## MLP Tuning

In [42]:
from re import X
from hybrid_mlp_model import HybridMLPClassifier

def mlp_objective(trial):
    params = {
        'feature_hidden_dim': trial.suggest_categorical('feature_hidden_dim', [16, 32, 64, 128]),
        'dropout': trial.suggest_float('dropout', 0.1, 0.5, step=0.1),
        'lr': trial.suggest_float('lr', 1e-4, 1e-2, log=True),
        'weight_decay': trial.suggest_float('weight_decay', 1e-5, 1e-3, log=True),
        'batch_size': trial.suggest_categorical('batch_size', [32, 64, 128]),
        'pos_weight': trial.suggest_float('pos_weight', 1.0, 10.0, step=0.5),
        'threshold': trial.suggest_float('threshold', 0.2, 0.5, step=0.05)
    }

    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    model = HybridMLPClassifier(
        embedding_dim=config["embedding_dim"],
        num_features=len(numeric_features),
        feature_hidden_dim=params['feature_hidden_dim'],
        dropout=params['dropout']
    ).to(device)
    
    train_loader = DataLoader(
        TensorDataset(
            torch.FloatTensor(X_emb_train),
            torch.FloatTensor(features_train),
            torch.FloatTensor(y_train)
        ),
        batch_size=params['batch_size'],
        shuffle=True
    )
    
    inner_val_loader = DataLoader(
        TensorDataset(
            torch.FloatTensor(X_emb_val),
            torch.FloatTensor(features_val),
            torch.FloatTensor(y_val)
        ),
        batch_size=params['batch_size'],
        shuffle=False
    )
    
    optimizer = torch.optim.Adam(
        model.parameters(),
        lr=params['lr'],
        weight_decay=params['weight_decay']
    )
    
    criterion = nn.BCEWithLogitsLoss(
        pos_weight=torch.tensor([params['pos_weight']], device=device)
    )
    
    best_val_loss = float('inf')
    patience = 5
    patience_counter = 0
    best_model_state = None
    
    for epoch in range(30):
        model.train()
        for embeddings, features, labels in train_loader:
            embeddings, features, labels = embeddings.to(device), features.to(device), labels.to(device)
            
            optimizer.zero_grad()
            logits = model(embeddings, features).squeeze()
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
        
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for embeddings, features, labels in inner_val_loader:
                embeddings, features, labels = embeddings.to(device), features.to(device), labels.to(device)
                logits = model(embeddings, features).squeeze()
                val_loss += criterion(logits, labels).item()
        
        val_loss /= len(inner_val_loader)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            best_model_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
        else:
            patience_counter += 1
            if patience_counter >= patience:
                break
    
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        model.to(device)
    
    val_loader = DataLoader(
        TensorDataset(
            torch.FloatTensor(X_emb_val),
            torch.FloatTensor(features_val),
            torch.FloatTensor(y_val)
        ),
        batch_size=params['batch_size'],
        shuffle=False
    )
    
    model.eval()
    val_probs = []
    val_labels = []
    
    with torch.no_grad():
        for embeddings, features, labels in val_loader:
            embeddings, features = embeddings.to(device), features.to(device)
            logits = model(embeddings, features).squeeze()
            probs = torch.sigmoid(logits)
            
            if probs.dim() == 0:
                val_probs.append(probs.cpu().item())
                val_labels.append(labels.item())
            else:
                val_probs.extend(probs.cpu().numpy())
                val_labels.extend(labels.numpy())
    
    val_probs = np.array(val_probs)
    val_labels = np.array(val_labels)
    val_preds = (val_probs >= params['threshold']).astype(int)
    
    recall = recall_score(val_labels, val_preds, zero_division=0)
    precision = precision_score(val_labels, val_preds, zero_division=0)
    f1 = f1_score(val_labels, val_preds, zero_division=0)
    roc_auc = roc_auc_score(val_labels, val_probs)
    
    trial.set_user_attr('recall', recall)
    trial.set_user_attr('precision', precision)
    trial.set_user_attr('f1_score', f1)
    trial.set_user_attr('roc_auc', roc_auc)
    
    if precision + recall > 0:
        F2 = (5 * precision * recall) / (4 * precision + recall)
    else:
        F2 = 0
    return F2

In [43]:
wandb_callback_mlp = WeightsAndBiasesCallback(
    metric_name="objective_score",
    wandb_kwargs={"project": "phishstop-detection", "tags": ["optuna", "mlp"]}
)

mlp_study = optuna.create_study(
    direction="maximize",
    study_name="mlp-tuning",
    sampler=optuna.samplers.TPESampler(seed=42)
)

mlp_study.optimize(
    mlp_objective,
    n_trials=config["n_trials_mlp"],
    callbacks=[wandb_callback_mlp],
    show_progress_bar=True
)

print(f"Best score: {mlp_study.best_trial.value:.4f}")
print(f"Recall: {mlp_study.best_trial.user_attrs['recall']:.4f}")
print(f"ROC-AUC: {mlp_study.best_trial.user_attrs['roc_auc']:.4f}")
print(f"Precision: {mlp_study.best_trial.user_attrs['precision']:.4f}")
print(f"F1: {mlp_study.best_trial.user_attrs['f1_score']:.4f}")
print("Best params:")
for k, v in mlp_study.best_trial.params.items():
    print(f"  {k}: {v}")

C:\Users\Latandu\AppData\Local\Temp\ipykernel_20948\1874426355.py:1: ExperimentalWarning:

WeightsAndBiasesCallback is experimental (supported from v2.9.0). The interface can change in the future.



batch_size,▁▃█▁▁█▃▃▃▁██████████████████▁▃
dropout,▁▃▅▁█▃▆█▁█▅▅▅▆▆▆▆▆▆▃█▆▆▆█▆▅▆█▅
feature_hidden_dim,▂▁▂▄▄▄▂▄▁▄██████████▁███████▁▂
lr,▁▂▁▂█▂▁▂▃▆▁▁▁▁▁▂▁▄▁▁▁▁▂▁▃▂▃▁▁▁
objective_score,▆▇▇▇▁▅▇▆▇▂▇█▇██▇█▅█▇▇█▇▇▆█▇███
pos_weight,▁▃█▃▁▆▃▆▄▇▅▅▅▅▆▆▄▆▂▅█▄▅▇▆▂▅▄▄▃
threshold,█▃▇▄▃▂▁▄▇▄▇▇▇███▆█▆▆█▆█▆▇▃█▆▆▄
weight_decay,▁▁▂▁▂▁▁▁▁▂▆▇██▄▄▄▄▅▃▃▅▅█▅▂▃▆▅▃
batch_size,64
dropout,0.3
feature_hidden_dim,32


[I 2025-12-01 02:36:49,796] A new study created in memory with name: mlp-tuning
Best trial: 0. Best value: 0.978797:   3%|▎         | 1/30 [02:06<1:00:54, 126.01s/it]

[I 2025-12-01 02:38:55,803] Trial 0 finished with value: 0.9787969839560067 and parameters: {'feature_hidden_dim': 32, 'dropout': 0.1, 'lr': 0.00020511104188433984, 'weight_decay': 1.3066739238053272e-05, 'batch_size': 32, 'pos_weight': 1.0, 'threshold': 0.5}. Best is trial 0 with value: 0.9787969839560067.


Best trial: 0. Best value: 0.978797:   7%|▋         | 2/30 [02:48<35:58, 77.10s/it]   

...

[I 2025-12-01 03:13:02,545] Trial 27 finished with value: 0.980467721923525 and parameters: {'feature_hidden_dim': 128, 'dropout': 0.4, 'lr': 0.0005959942777273677, 'weight_decay': 8.100407637142324e-05, 'batch_size': 128, 'pos_weight': 4.0, 'threshold': 0.45}. Best is trial 19 with value: 0.9819457807459393.


Best trial: 19. Best value: 0.981946:  97%|█████████▋| 29/30 [37:07<01:04, 64.24s/it]

[I 2025-12-01 03:13:57,646] Trial 28 finished with value: 0.9792118447618336 and parameters: {'feature_hidden_dim': 16, 'dropout': 0.5, 'lr': 0.0018836929731922672, 'weight_decay': 0.0002626045912808322, 'batch_size': 128, 'pos_weight': 3.0, 'threshold': 0.4}. Best is trial 19 with value: 0.9819457807459393.


Best trial: 19. Best value: 0.981946: 100%|██████████| 30/30 [40:24<00:00, 80.82s/it] 

[I 2025-12-01 03:17:14,329] Trial 29 finished with value: 0.9777106157914619 and parameters: {'feature_hidden_dim': 32, 'dropout': 0.4, 'lr': 0.0002452510912989698, 'weight_decay': 0.0005475202356640898, 'batch_size': 32, 'pos_weight': 1.5, 'threshold': 0.5}. Best is trial 19 with value: 0.9819457807459393.
Best score: 0.9819
Recall: 0.9846
ROC-AUC: 0.9982
Precision: 0.9715
F1: 0.9780
Best params:
  feature_hidden_dim: 128
  dropout: 0.5
  lr: 0.0005957412292147004
  weight_decay: 0.00023686574806450268
  batch_size: 128
  pos_weight: 2.0
  threshold: 0.4


In [44]:
fig = optuna.visualization.plot_optimization_history(mlp_study)
fig.update_layout(width=800, height=500) 
fig.show()

fig = optuna.visualization.plot_param_importances(mlp_study)
fig.update_layout(width=800, height=500) 
fig.show()

## Save Results

In [45]:
results = {
    'tfidf': {
        'best_score': tfidf_study.best_trial.value,
        'recall': tfidf_study.best_trial.user_attrs['recall'],
        'roc_auc': tfidf_study.best_trial.user_attrs['roc_auc'],
        'precision': tfidf_study.best_trial.user_attrs['precision'],
        'f1_score': tfidf_study.best_trial.user_attrs['f1_score'],
        'best_params': tfidf_study.best_trial.params
    },
    'xgboost': {
        'best_score': xgb_study.best_trial.value,
        'recall': xgb_study.best_trial.user_attrs['recall'],
        'roc_auc': xgb_study.best_trial.user_attrs['roc_auc'],
        'precision': xgb_study.best_trial.user_attrs['precision'],
        'f1_score': xgb_study.best_trial.user_attrs['f1_score'],
        'best_params': xgb_study.best_trial.params
    },
    'mlp': {
        'best_score': mlp_study.best_trial.value,
        'recall': mlp_study.best_trial.user_attrs['recall'],
        'roc_auc': mlp_study.best_trial.user_attrs['roc_auc'],
        'precision': mlp_study.best_trial.user_attrs['precision'],
        'f1_score': mlp_study.best_trial.user_attrs['f1_score'],
        'best_params': mlp_study.best_trial.params
    }
}

os.makedirs("../../output/tuning_results", exist_ok=True)

with open('../../output/tuning_results/optuna_results.json', 'w') as f:
    json.dump(results, f, indent=2)

for model, res in results.items():
    print(f"{model.upper()}:")
    print(f"Score F2 = (5 * precision * recall) / (4 * precision + recall): {res['best_score']:.4f}")
    print(f"Recall: {res['recall']:.4f}")
    print(f"ROC-AUC: {res['roc_auc']:.4f}")
    print(f"Precision: {res['precision']:.4f}")
    print(f"F1 Score: {res['f1_score']:.4f}")

TFIDF:
Score F2 = (5 * precision * recall) / (4 * precision + recall): 0.9790
Recall: 0.9925
ROC-AUC: 0.9973
Precision: 0.9285
F1 Score: 0.9595
XGBOOST:
Score F2 = (5 * precision * recall) / (4 * precision + recall): 0.9770
Recall: 0.9833
ROC-AUC: 0.9974
Precision: 0.9530
F1 Score: 0.9679
MLP:
Score F2 = (5 * precision * recall) / (4 * precision + recall): 0.9819
Recall: 0.9846
ROC-AUC: 0.9982
Precision: 0.9715
F1 Score: 0.9780
